In [1]:
import string
from typing import Optional
from dataclasses import dataclass
from collections import defaultdict
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import cached_property
from math import log

from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

np.random.seed(42)

## Расчет метрик

In [2]:
validation_query_positives = pd.read_parquet("validation_query_positives.parquet")

In [3]:
validation_query_positives.head()

,query,products
0,вафли,"[400888832, 264708097, 184276484, 941928453, 1..."
1,минеральная,"[140030724, 148001285, 1147920517, 1211305607,..."
2,минеральная вода газированная,"[1318867975, 148001288, 148001289, 1318867977,..."
3,фруктовые пюре детские,"[146804864, 146804992, 146804995, 282172038, 1..."
4,хлеб тостовый,"[612105734, 756423174, 1326541838, 266528783, ..."


In [4]:
validation_query_positives_dict = {
    row[1].query: set(row[1].products.tolist()) for row in validation_query_positives.iterrows()
}

In [5]:
@dataclass
class Metrics:
    precision: float
    recall: float
    f1_score: float
        
    def __repr__(self):
        return f"precision = {self.precision}\nrecall = {self.recall}\nf1_score = {self.f1_score}"


In [6]:
def calculate_metrics(ground_truth_set, search_results_set):
    
    # True positives: items that are both in ground truth and search results
    tp = len(ground_truth_set.intersection(search_results_set))
    
    # Precision: tp / (tp + fp)
    precision = tp / len(search_results_set) if len(search_results_set) > 0 else 0.0
    
    # Recall: tp / (tp + fn)
    recall = tp / len(ground_truth_set) if len(ground_truth_set) > 0 else 0.0
    
    # F1-score: harmonic mean of precision and recall
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return Metrics(precision=precision, recall=recall, f1_score=f1_score)


In [7]:
def calculate_validation_metrics(search_function, limit=20):
    metrics = []
    for query, positives in validation_query_positives_dict.items():
        metrics.append(
            calculate_metrics(positives, search_function(query=query, limit=limit))
        )
    
    return Metrics(
        precision=np.mean([x.precision for x in metrics]),
        recall=np.mean([x.recall for x in metrics]),
        f1_score=np.mean([x.f1_score for x in metrics]),
    )
    

## Обработка документов

In [8]:
dataset = pd.read_parquet("products_with_names.parquet")

In [9]:
dataset.head()

,product_id,name
0,4036767,"Модуль сменный фильтрующий Аквафор КН, 208731"
1,4050873,"Водоочиститель Аквафор модель Кристалл Н, 2059..."
2,4226160,Развиваем мышление (2-3 года) | Земцова Ольга
3,4644911,Lacoste Вода парфюмерная Pour Femme 50 мл
4,4788809,Сменные Кассеты Для Мужской Бритвы Gillette Ma...


In [10]:
documents_dict = {
    doc[1]["product_id"]: doc[1]["name"] for doc in dataset.iterrows()
}

In [11]:
@dataclass
class Document:
    doc_id: int
    name: str

documents = [Document(doc_id=doc[1]["product_id"], name=doc[1]["name"]) for doc in dataset.iterrows()]


In [12]:
class TextProcessor:
    def __init__(self):
        self.symbols_to_replace = {"ё": "е"}
        self.stopwords = set(stopwords.words("russian"))
        self.linguist = MorphAnalyzer()

    def lowercase_text(self, text: str) -> str:
        return text.lower()

    def replace_symbols(self, text: str) -> str:
        for old, new in self.symbols_to_replace.items():
            text = text.replace(old, new)
        return text

    def process_punctuation_simple(self, text: str) -> str:
        translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
        text_without_punc = text.translate(translation_table)
        text_without_double_spaces = ' '.join(text_without_punc.split())
        return text_without_double_spaces

    def tokenize_simple(self, text: str) -> list[str]:
        return text.split()

    def remove_stopwords(self, doc: list[str]) -> list[str]:
        return [token for token in doc if token not in self.stopwords]

    def lemmatize_token(self, token: str) -> str:
        return self.linguist.normal_forms(token)[0]
    
    def lemmatize_tokenized_text(self, tokenized_text: list[str]) -> list[str]:
        return [self.lemmatize_token(token) for token in tokenized_text]

    def process_text(self, text: str) -> list[str]:
        text = self.lowercase_text(text)
        text = self.replace_symbols(text)
        text = self.process_punctuation_simple(text)
        text_tokens = self.tokenize_simple(text)
        text_tokens = self.remove_stopwords(text_tokens)
        return self.lemmatize_tokenized_text(text_tokens)
        

In [13]:
text_processor = TextProcessor()

In [14]:
documents_processed = [(document.doc_id, text_processor.process_text(document.name)) for document in tqdm(documents)]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 238443/238443 [02:30<00:00, 1580.98it/s]


In [15]:
def print_ozon_link(product_id: int) -> str:
    print(f"https://ozon.ru/product/{product_id}")

## Абстрактный поисковый движок

In [17]:
from abc import ABC, abstractmethod


class SearchEngine(ABC):
    @abstractmethod
    def index_documents(self, documents_processed: list[tuple[int, list[str]]]) -> None: ...

    @abstractmethod
    def search(self, query: str, limit: Optional[int] = None) -> dict[int, float]: ...


## Поиск по обратному индексу с ранжированием по term frequency

In [18]:
class SearchEngineTf(SearchEngine):
    def __init__(self, text_processor: TextProcessor):
        super().__init__()
        self._index: dict[str, dict[int, int]] = defaultdict(lambda: defaultdict(int))
        self._documents: set[int] = set()
        self._text_processor = text_processor

    def index_documents(self, documents_processed: list[tuple[int, list[str]]]) -> None:
        for doc_id, name in documents_processed:
            self.index(doc_id, name)

    def index(self, doc_id: int, name: list[str]) -> None:
        self._documents.add(doc_id)
        for token in name:
            self._index[token][doc_id] += 1

    def search(self, query: str, limit: Optional[int] = None) -> dict[int, float]:
        query_processed = self._text_processor.process_text(query)
        product_scores: dict[int, float] = dict()
        for query_token in query_processed:
            query_token_product_scores = self._index[query_token]
            product_scores = self.update_product_scores(product_scores, query_token_product_scores)
        return [doc[0] for doc in sorted(product_scores.items(), key=lambda x: x[1], reverse=True)][:limit]

    def update_product_scores(self, old: dict[int, float], new: dict[int, float]) -> dict[int, float]:
        for doc_id, score in new.items():
            if doc_id in old:
                old[doc_id] += score
            else:
                old[doc_id] = score
        return old


In [19]:
search_engine_tf = SearchEngineTf(text_processor)

In [20]:
search_engine_tf.index_documents(documents_processed)

In [21]:
calculate_validation_metrics(search_engine_tf.search, limit=200)

precision = 0.13169453942460838
recall = 0.18880294358930627
f1_score = 0.13554963302537784

## Поиск по обратному индексу с ранжированием по tf-idf

In [22]:
class SearchEngineTfIdf(SearchEngine):
    def __init__(self, text_processor: TextProcessor):
        super().__init__()
        self._index: dict[str, dict[int, int]] = defaultdict(lambda: defaultdict(int))
        self._documents: set[int] = set()
        self._text_processor = text_processor

    def index_documents(self, documents_processed: list[tuple[int, list[str]]]) -> None:
        for doc_id, name in documents_processed:
            self.index(doc_id, name)

    def index(self, doc_id: int, name: list[str]) -> None:
        self._documents.add(doc_id)
        for token in name:
            self._index[token][doc_id] += 1

    @property
    def number_of_documents(self) -> int:
        return len(self._documents)

    def idf(self, query_token: str) -> float:
        N = self.number_of_documents
        n_query_token_docs = len(self._index[query_token])
        return log((N - n_query_token_docs + 0.5) / (n_query_token_docs + 0.5) + 1)

    def _get_query_token_document_scores(self, query_token: str) -> dict[int, float]:
        idf_score = self.idf(query_token)
        return {doc_id: tf * idf_score for doc_id, tf in self._index[query_token].items()}

    def search(self, query: str, limit: Optional[int] = None) -> dict[int, float]:
        query_processed = self._text_processor.process_text(query)
        product_scores: dict[int, float] = dict()
        for query_token in query_processed:
            query_token_product_scores = self._get_query_token_document_scores(query_token)
            product_scores = self.update_product_scores(product_scores, query_token_product_scores)
        return [doc[0] for doc in sorted(product_scores.items(), key=lambda x: x[1], reverse=True)][:limit]

    def update_product_scores(self, old: dict[int, float], new: dict[int, float]) -> dict[int, float]:
        for doc_id, score in new.items():
            if doc_id in old:
                old[doc_id] += score
            else:
                old[doc_id] = score
        return old


In [23]:
search_engine_tf_idf = SearchEngineTfIdf(text_processor)

In [24]:
search_engine_tf_idf.index_documents(documents_processed)

In [25]:
calculate_validation_metrics(search_engine_tf_idf.search, limit=200)

precision = 0.13344453942460838
recall = 0.19305872757962741
f1_score = 0.13788085205734998

## Поиск по обратному индексу с ранжированием по bm25

In [33]:
class SearchEngineBm25(SearchEngine):
    def __init__(self, text_processor: TextProcessor, k1: float = 1.2, b: float = 0.75):
        super().__init__()
        self._index: dict[str, dict[int, int]] = defaultdict(lambda: defaultdict(int))
        self._documents: dict[int, list[str]] = dict()
        self._text_processor = text_processor
        self.k1 = k1
        self.b = b

    def index_documents(self, documents_processed: list[tuple[int, list[str]]]) -> None:
        for doc_id, name in documents_processed:
            self.index(doc_id, name)

    def index(self, doc_id: int, name: list[str]) -> None:
        self._documents[doc_id] = name
        for token in name:
            self._index[token][doc_id] += 1

    @cached_property
    def number_of_documents(self) -> int:
        return len(self._documents)

    def idf(self, query_token: str) -> float:
        N = self.number_of_documents
        n_query_token_docs = len(self._index[query_token])
        return log((N - n_query_token_docs + 0.5) / (n_query_token_docs + 0.5) + 1)

    @cached_property
    def avdl(self) -> float:
        return sum(len(doc) for doc in self._documents.values()) / self.number_of_documents

    def _get_query_token_document_scores(self, query_token: str) -> dict[int, float]:
        idf_score = self.idf(query_token)
        return {
            doc_id: idf_score * tf * (self.k1 + 1) / (tf + self.k1 * (1 - self.b + self.b * (len(self._documents[doc_id]) / self.avdl)))
            for doc_id, tf in self._index[query_token].items()
        }

    def search(self, query: str, limit: Optional[int] = None) -> dict[int, float]:
        query_processed = self._text_processor.process_text(query)
        product_scores: dict[int, float] = dict()
        for query_token in query_processed:
            query_token_product_scores = self._get_query_token_document_scores(query_token)
            product_scores = self.update_product_scores(product_scores, query_token_product_scores)
        return [doc[0] for doc in sorted(product_scores.items(), key=lambda x: x[1], reverse=True)][:limit]

    def update_product_scores(self, old: dict[int, float], new: dict[int, float]) -> dict[int, float]:
        for doc_id, score in new.items():
            if doc_id in old:
                old[doc_id] += score
            else:
                old[doc_id] = score
        return old


In [34]:
search_engine_bm25 = SearchEngineBm25(text_processor, k1=1.2, b=0.75)

In [35]:
search_engine_bm25.index_documents(documents_processed)

In [36]:
calculate_validation_metrics(search_engine_bm25.search, limit=200)

precision = 0.1547195394246084
recall = 0.2418148590206131
f1_score = 0.16617490389624023

## Поиск по обратному индексу на n-gram-ах с ранжированием по BM25 

In [50]:
from collections import defaultdict
from typing import List, Tuple, Optional, Dict, Set
import itertools

class SearchEngineNgramBm25(SearchEngine):
    def __init__(self, text_processor: TextProcessor, k1: float = 1.2, b: float = 0.75, n_grams_size: int = 3):
        super().__init__()
        self._index: dict[str, dict[int, int]] = defaultdict(lambda: defaultdict(int))
        self._documents: dict[int, list[str]] = dict()
        self._text_processor = text_processor
        self.k1 = k1
        self.b = b
        self._n_grams_size = n_grams_size

    def index_documents(self, documents: list[tuple[int, str]]) -> None:
        for doc in documents:
            self.index(doc.doc_id, doc.name)

    def index(self, doc_id: int, name: list[str]) -> None:
        doc_name_processed = self._text_processor.process_text(name)
        ngrams = self._generate_ngrams(doc_name_processed)
        self._documents[doc_id] = ngrams
        for ngram in ngrams:
            self._index[ngram][doc_id] += 1

    def _generate_ngrams(self, doc_name: str) -> list[str]:
        ngrams = []
        for i in range(len(doc_name) - self._n_grams_size + 1):
            ngrams.append(doc_name[i: i + self._n_grams_size])
        return ngrams

    @cached_property
    def number_of_documents(self) -> int:
        return len(self._documents)

    def idf(self, query_token: str) -> float:
        N = self.number_of_documents
        n_query_token_docs = len(self._index[query_token])
        return log((N - n_query_token_docs + 0.5) / (n_query_token_docs + 0.5) + 1)

    @cached_property
    def avdl(self) -> float:
        return sum(len(doc) for doc in self._documents.values()) / self.number_of_documents

    def _get_query_token_document_scores(self, query_token: str) -> dict[int, float]:
        idf_score = self.idf(query_token)
        return {
            doc_id: idf_score * tf * (self.k1 + 1) / (tf + self.k1 * (1 - self.b + self.b * (len(self._documents[doc_id]) / self.avdl)))
            for doc_id, tf in self._index[query_token].items()
        }

    def search(self, query: str, limit: Optional[int] = None) -> dict[int, float]:
        query_ngrams = self._generate_ngrams(self._text_processor.process_text(query))
        product_scores: dict[int, float] = dict()
        for query_ngram in query_ngrams:
            query_ngram_product_scores = self._get_query_token_document_scores(query_ngram)
            product_scores = self.update_product_scores(product_scores, query_ngram_product_scores)
        return [doc[0] for doc in sorted(product_scores.items(), key=lambda x: x[1], reverse=True)][:limit]

    def update_product_scores(self, old: dict[int, float], new: dict[int, float]) -> dict[int, float]:
        for doc_id, score in new.items():
            if doc_id in old:
                old[doc_id] += score
            else:
                old[doc_id] = score
        return old

In [51]:
class SimpleTextProcessor(TextProcessor):
    def process_text(self, text: str) -> str:
        text = self.lowercase_text(text)
        text = self.replace_symbols(text)
        return self.process_punctuation_simple(text)
        

In [52]:
simple_text_processor = SimpleTextProcessor()

In [53]:
search_engine_ngram_bm25 = SearchEngineNgramBm25(simple_text_processor, k1=1.2, b=0.75, n_grams_size=3)

In [54]:
search_engine_ngram_bm25.index_documents(documents)

In [55]:
[print_ozon_link(x) for x in search_engine_ngram_bm25.search("кассеты для бритвы мужские", limit=30)]
print()

https://ozon.ru/product/465164863
https://ozon.ru/product/835428859
https://ozon.ru/product/482126805
https://ozon.ru/product/591928165
https://ozon.ru/product/680978604
https://ozon.ru/product/620047873
https://ozon.ru/product/620046931
https://ozon.ru/product/746671114
https://ozon.ru/product/168165795
https://ozon.ru/product/630691239
https://ozon.ru/product/217068283
https://ozon.ru/product/161674518
https://ozon.ru/product/517209910
https://ozon.ru/product/168894035
https://ozon.ru/product/267981985
https://ozon.ru/product/184573224
https://ozon.ru/product/640910116
https://ozon.ru/product/164214504
https://ozon.ru/product/4788808
https://ozon.ru/product/144505497
https://ozon.ru/product/669972122
https://ozon.ru/product/669985694
https://ozon.ru/product/669970989
https://ozon.ru/product/670058352
https://ozon.ru/product/670058897
https://ozon.ru/product/670058654
https://ozon.ru/product/161674517
https://ozon.ru/product/4788809
https://ozon.ru/product/144505500
https://ozon.ru/pr

In [56]:
[print_ozon_link(x) for x in search_engine_bm25.search("кассеты для бритвы мужские", limit=30)]
print()

https://ozon.ru/product/465164863
https://ozon.ru/product/835428859
https://ozon.ru/product/482126805
https://ozon.ru/product/517209910
https://ozon.ru/product/217068283
https://ozon.ru/product/168165795
https://ozon.ru/product/231812359
https://ozon.ru/product/640910116
https://ozon.ru/product/184573224
https://ozon.ru/product/580637093
https://ozon.ru/product/216665672
https://ozon.ru/product/746671114
https://ozon.ru/product/164214504
https://ozon.ru/product/4788808
https://ozon.ru/product/620047873
https://ozon.ru/product/583952991
https://ozon.ru/product/620046931
https://ozon.ru/product/157424893
https://ozon.ru/product/5543624
https://ozon.ru/product/138423967
https://ozon.ru/product/626512687
https://ozon.ru/product/1387337779
https://ozon.ru/product/811011625
https://ozon.ru/product/239779174
https://ozon.ru/product/356922446
https://ozon.ru/product/4788809
https://ozon.ru/product/4789070
https://ozon.ru/product/1462804333
https://ozon.ru/product/162865256
https://ozon.ru/prod

In [57]:
[print_ozon_link(x) for x in search_engine_ngram_bm25.search("косеты для бритвы мужские", limit=30)]
print()

https://ozon.ru/product/465164863
https://ozon.ru/product/835428859
https://ozon.ru/product/482126805
https://ozon.ru/product/620047873
https://ozon.ru/product/620046931
https://ozon.ru/product/680978604
https://ozon.ru/product/591928165
https://ozon.ru/product/746671114
https://ozon.ru/product/168165795
https://ozon.ru/product/217068283
https://ozon.ru/product/630691239
https://ozon.ru/product/669972122
https://ozon.ru/product/669985694
https://ozon.ru/product/669970989
https://ozon.ru/product/670058352
https://ozon.ru/product/267981985
https://ozon.ru/product/184573224
https://ozon.ru/product/670058897
https://ozon.ru/product/670058654
https://ozon.ru/product/640910116
https://ozon.ru/product/161674518
https://ozon.ru/product/517209910
https://ozon.ru/product/4788809
https://ozon.ru/product/144505500
https://ozon.ru/product/4788804
https://ozon.ru/product/164214502
https://ozon.ru/product/162865257
https://ozon.ru/product/4788805
https://ozon.ru/product/162865256
https://ozon.ru/prod

In [58]:
[print_ozon_link(x) for x in search_engine_bm25.search("косеты для бритвы мужские", limit=30)]
print()

https://ozon.ru/product/1387337779
https://ozon.ru/product/356922446
https://ozon.ru/product/648028268
https://ozon.ru/product/790115501
https://ozon.ru/product/208009731
https://ozon.ru/product/714380855
https://ozon.ru/product/482126805
https://ozon.ru/product/1416463253
https://ozon.ru/product/1048144951
https://ozon.ru/product/465164863
https://ozon.ru/product/847166922
https://ozon.ru/product/1344582380
https://ozon.ru/product/835428859
https://ozon.ru/product/683899493
https://ozon.ru/product/208009732
https://ozon.ru/product/1144916855
https://ozon.ru/product/226988572
https://ozon.ru/product/1395866388
https://ozon.ru/product/517209910
https://ozon.ru/product/1579101798
https://ozon.ru/product/669985722
https://ozon.ru/product/1340978156
https://ozon.ru/product/217068283
https://ozon.ru/product/284926105
https://ozon.ru/product/1409858565
https://ozon.ru/product/596835379
https://ozon.ru/product/245755783
https://ozon.ru/product/168165795
https://ozon.ru/product/275803956
https

In [59]:
[print_ozon_link(x) for x in search_engine_bm25.search("халодильник", limit=30)]
print()

In [60]:
[print_ozon_link(x) for x in search_engine_ngram_bm25.search("халодильник", limit=30)]
print()

https://ozon.ru/product/209997102
https://ozon.ru/product/1576425147
https://ozon.ru/product/1457779141
https://ozon.ru/product/1444369749
https://ozon.ru/product/193470153
https://ozon.ru/product/170075383
https://ozon.ru/product/193470154
https://ozon.ru/product/182451989
https://ozon.ru/product/182451991
https://ozon.ru/product/154475460
https://ozon.ru/product/266791724
https://ozon.ru/product/1075792425
https://ozon.ru/product/170075379
https://ozon.ru/product/914168984
https://ozon.ru/product/154072681
https://ozon.ru/product/266791176
https://ozon.ru/product/914170197
https://ozon.ru/product/1074359967
https://ozon.ru/product/170075380
https://ozon.ru/product/170075385
https://ozon.ru/product/170075384
https://ozon.ru/product/170075378
https://ozon.ru/product/154474648
https://ozon.ru/product/170075382
https://ozon.ru/product/154073300
https://ozon.ru/product/170075381
https://ozon.ru/product/914170194
https://ozon.ru/product/415182919
https://ozon.ru/product/415173072
https://o

In [61]:
[print_ozon_link(x) for x in search_engine_bm25.search("новушники", limit=30)]
print()

In [62]:
[print_ozon_link(x) for x in search_engine_ngram_bm25.search("новушники", limit=30)]
print()

https://ozon.ru/product/220484272
https://ozon.ru/product/544315175
https://ozon.ru/product/202774783
https://ozon.ru/product/1549417381
https://ozon.ru/product/645008371
https://ozon.ru/product/505543662
https://ozon.ru/product/793717993
https://ozon.ru/product/254052997
https://ozon.ru/product/254070112
https://ozon.ru/product/294968666
https://ozon.ru/product/294969778
https://ozon.ru/product/254051714
https://ozon.ru/product/190759201
https://ozon.ru/product/294968551
https://ozon.ru/product/294969794
https://ozon.ru/product/190759208
https://ozon.ru/product/779662322
https://ozon.ru/product/407158738
https://ozon.ru/product/589737338
https://ozon.ru/product/1540354338
https://ozon.ru/product/698850642
https://ozon.ru/product/465680921
https://ozon.ru/product/1380001456
https://ozon.ru/product/273220354
https://ozon.ru/product/1643701055
https://ozon.ru/product/589776489
https://ozon.ru/product/964575571
https://ozon.ru/product/310586499
https://ozon.ru/product/701052713
https://oz

In [63]:
calculate_validation_metrics(search_engine_ngram_bm25.search, limit=200)

precision = 0.117575
recall = 0.2518484929482551
f1_score = 0.15226755762491367